<a href="https://colab.research.google.com/github/Sebukpor/project-sebukpor/blob/main/predicting_effector_angles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import sklearn
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [48]:
import pandas as pd

# Define column names
column_names = ['JointAngle1', 'JointAngle2', 'JointAngle3', 'JointAngle4', 'JointAngle5', 'JointAngle6',
                'X_Position', 'Y_Position', 'Z_Position', 'RotationElement1', 'RotationElement2', 'RotationElement3']

# Load the dataset with specified column names
data = pd.read_csv('/content/random.csv', names=column_names)

# Display the first few rows of the dataset to verify column names
print(data.head())

                                                    JointAngle1  JointAngle2  \
-0.88254 2.1659  1.25370  3.82890  0.37542 -5.4401    -0.516810      0.88886   
 0.63053 1.6022  1.61690  2.36320 -1.04630 -1.7015    -0.664540      0.05001   
 0.85788 2.5143  2.47530  0.85839 -0.53443 -5.4392    -2.069200     -1.97410   
 0.68397 2.8924 -0.15305 -3.94820 -0.81534  5.4264    -0.716570     -0.10199   
 2.24920 2.5581 -1.10640  2.13620  1.87980  2.7333     0.069533      0.82531   

                                                    JointAngle3  JointAngle4  \
-0.88254 2.1659  1.25370  3.82890  0.37542 -5.4401      4.00790      0.79437   
 0.63053 1.6022  1.61690  2.36320 -1.04630 -1.7015      3.95280     -0.20176   
 0.85788 2.5143  2.47530  0.85839 -0.53443 -5.4392      0.96334      0.44964   
 0.68397 2.8924 -0.15305 -3.94820 -0.81534  5.4264      3.22820      0.51233   
 2.24920 2.5581 -1.10640  2.13620  1.87980  2.7333      1.15990     -0.49078   

                                      

In [38]:
# Create a StandardScaler object
scaler = StandardScaler()

In [39]:
X_scaled = scaler.fit_transform(X)
Y_scaled = scaler.fit_transform(Y)

In [41]:
print(X_scaled)

[[-0.41884726  0.6585858   1.42120276 ...  0.33867712  0.22694369
   1.55316382]
 [-0.53354971  0.04455869  1.37926961 ... -1.53773351  0.17735593
   0.64429407]
 [-1.62417412 -1.43706308 -0.89582058 ... -1.54925497 -0.76971332
   0.00404532]
 ...
 [-0.33567576 -0.38656689  0.8221904  ... -0.63911249 -1.55431491
  -0.56143603]
 [-0.1236319  -0.26515224  1.58246653 ... -0.69595282 -0.57902569
   1.34423948]
 [-0.27289338 -0.7380157  -1.597076   ...  1.3538548   1.08940479
  -0.17174291]]


In [42]:
print(Y_scaled)

[[-1.00433649  1.4170759  -0.02485037]
 [ 0.70032948 -0.44121524  1.48870839]
 [ 0.04365398  0.19376337  1.68963776]
 ...
 [-1.16727582  0.74970759 -1.05881055]
 [-0.82134045  1.51838808  0.1804987 ]
 [ 0.9035708  -1.14002526  0.92675594]]


In [43]:
mean_values = scaler.mean_
scale_values = scaler.scale_

print("Mean Values:", mean_values)
print("Scale Values:", scale_values)

Mean Values: [0.00911563 0.00970745 0.01443243]
Scale Values: [0.58334595 0.56962549 0.57866106]


In [44]:
# Define your neural network model with a name
model = tf.keras.Sequential(name="my_end_effector_prediction")

# Determine the input size based on the number of features in your dataset
input_size = X_scaled.shape[1]  # Use X_scaled shape
print("Input size:", input_size)

# Add layers to the model
model.add(tf.keras.layers.Dense(units=128, input_shape=(input_size,), activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))  # Adjust dropout rate
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))  # Adjust dropout rate
model.add(tf.keras.layers.Dense(units=3, activation='linear'))  # Use linear activation for regression

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use mean squared error for regression
              metrics=['mae'])  # Monitor mean absolute error

# Fit the model to the training data
history = model.fit(X_scaled, Y_scaled, epochs=30, batch_size=64, validation_split=0.2)  # Adjust epochs and batch size

# Print the training history
print(history.history.keys())

# Print the final training and validation accuracy
train_mae = history.history['mae'][-1]
val_mae = history.history['val_mae'][-1]
print("Final training MAE:", train_mae)
print("Final validation MAE:", val_mae)

# Save the model to a file
model.save('my_end_effector_prediction')


Input size: 9
Epoch 1/30
63/63 [==============================] - 1s 6ms/step - loss: 0.6894 - mae: 0.6906 - val_loss: 0.1933 - val_mae: 0.3636
Epoch 2/30
63/63 [==============================] - 0s 4ms/step - loss: 0.2538 - mae: 0.4034 - val_loss: 0.0500 - val_mae: 0.1767
Epoch 3/30
63/63 [==============================] - 0s 5ms/step - loss: 0.1854 - mae: 0.3425 - val_loss: 0.0341 - val_mae: 0.1519
Epoch 4/30
63/63 [==============================] - 0s 5ms/step - loss: 0.1554 - mae: 0.3138 - val_loss: 0.0269 - val_mae: 0.1367
Epoch 5/30
63/63 [==============================] - 0s 4ms/step - loss: 0.1350 - mae: 0.2943 - val_loss: 0.0211 - val_mae: 0.1204
Epoch 6/30
63/63 [==============================] - 0s 3ms/step - loss: 0.1273 - mae: 0.2843 - val_loss: 0.0162 - val_mae: 0.1033
Epoch 7/30
63/63 [==============================] - 0s 4ms/step - loss: 0.1194 - mae: 0.2753 - val_loss: 0.0206 - val_mae: 0.1198
Epoch 8/30
63/63 [==============================] - 0s 3ms/step - loss: 0.11

In [46]:
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('my_end_effector_prediction')

# Function to preprocess user input
def preprocess_input(input_data):
    # Perform any necessary preprocessing, such as scaling or normalization
    # Make sure to apply the same preprocessing steps used during training

    # If data is already preprocessed
    scaled_input = input_data
    return scaled_input

# Function to predict end-effector positions
def predict_end_effector_position(input_data):
    # Preprocess the input data
    scaled_input = preprocess_input(input_data)

    # Make predictions using the trained model
    predictions = model.predict(scaled_input)

    # Return the predicted end-effector positions
    return predictions

# input joint angles and rotation effect to predict your end-effector postions
if __name__ == "__main__":
    # Adjusting  the input shape to match the model's input size (9 features)
    user_input =np.array([[0.88254, 2.1659, -0.23327, -0.57676, 0.81691, 5.25E-05, 0.19059, 0.1345, 0.97241]])


    # Predict end-effector positions
    predicted_positions = predict_end_effector_position(user_input)

    # Print the predicted positions
    print("Predicted end-effector positions based on your inputs :")
    print(predicted_positions)


1/1 [==============================] - 0s 65ms/step
Predicted end-effector positions:
[[-0.8041317  -0.63412154  0.14666814]]
